# Spatial Join to HQTA

* Which HQTA file to use? Can use polygons first, and filter out the ferry and rail ones.
* Seems like we don't have that many facilities to retrofit anyway.
* Don't overcomplicate and think about whether it's near a bus's origin or destination.
* Start with overall, within a district, a map showing both facility locations (points) and HQTA polygons.
* In DTLA, that Caltrans building could be retrofitted, and the corridors it would serve are numerous.

Folium ideas:
* Use GeoJSON works, can use multiple layers function, but can't control marker
* Using Marker or CircleMarker gets it to look nicer, but output is way larger for html
* https://github.com/python-visualization/folium/issues/1059
* https://github.com/python-visualization/folium/pull/957
* look at this script: https://github.com/jtbaker/folium/blob/83745caa3613aede3d435909a325e13f9c50dde9/folium/features.py
* ex: https://github.com/jtbaker/folium/blob/83745caa3613aede3d435909a325e13f9c50dde9/examples/GeoJSONMarker.ipynb


In [ ]:
import geopandas as gpd
import intake
import pandas as pd

import utils
from shared_utils import geography_utils

catalog = intake.open_catalog("./*.yml")

In [ ]:
facilities = catalog.tier1_facilities_processed.read()

In [ ]:
hqta = catalog.hqta_shapes.read()

exclude_me = ["major_stop_ferry", 
           "major_stop_rail",
          ]

hqta = hqta[~hqta.hqta_type.isin(exclude_me)]

In [ ]:
hqta_cols = ['calitp_itp_id_primary', 'agency_name_primary', 'hqta_type',
       'calitp_itp_id_secondary', 'agency_name_secondary',]
hqta2 = hqta.dissolve(by=hqta_cols).reset_index()

In [ ]:
# spatial join
gdf = gpd.sjoin(
    facilities.to_crs(geography_utils.WGS84),
    hqta2.to_crs(geography_utils.WGS84),
    how = "inner",
    predicate = "intersects",
).drop(columns = "index_right").drop_duplicates()

In [ ]:
gdf.sheet_uuid.nunique()

In [ ]:
# Which locations
facility_cols = ["sqft", "category", 
                 "facility_name", "facility_type",
                 "address_arcgis_clean", 
                 "county_name", "district",
                ]
facilities = gdf[["sheet_uuid", "geometry"] + facility_cols].drop_duplicates()

# Just draw a small buffer around the point for now
# Then use function to plot multiple polygon layers
# Tweak that map_utils multiple layers function to take points?
'''
facilities = facilities.assign(
    longitude = facilities.geometry.x,
    latitude = facilities.geometry.y,
    geometry = (facilities.to_crs(geography_utils.CA_StatePlane).buffer(25)
                .to_crs(geography_utils.WGS84)
               )
)
'''

In [ ]:
# Which HQTA corridors (polygon geom)
hqta_cols = [
    'calitp_itp_id_primary',
    'agency_name_primary', 
    'hqta_type', 
    'calitp_itp_id_secondary',
    'agency_name_secondary'
]

hqta_corr = (gdf[["sheet_uuid"] + facility_cols + hqta_cols].drop_duplicates()
             .merge(hqta2,
                    on = hqta_cols,
                    how = "inner"
                   )
            )

hqta_corr = gpd.GeoDataFrame(hqta_corr).to_crs(geography_utils.WGS84)

In [ ]:
#facilities.to_parquet("./data/plot_facilities.parquet")
#hqta_corr.to_parquet("./data/plot_hqta.parquet")

In [ ]:
# A lot of these are in D4, D7, and a bunch that's HQ (subset of D3 geographically)
# Can do statewide map, but district stats
# For district, show breakdown of facility category / list of facility names

## Folium Map

Rewrite multiple layers function: 

* return folium.GeoJson object in sub-functions
* this should handle multiple polygon layers or a point layer with a polygon layers. check if line is handled...maybe lines should always get buffers drawn around them.
* unpack the list of GeoJson for multiple layers and add them to the Map, then add to Figure (`.add_to(m)`, `fig.add_child(m)`)

In [ ]:
import branca
import folium
import geopandas as gpd
import intake
import pandas as pd

import utils
import make_map
from shared_utils import map_utils
from shared_utils import calitp_color_palette as cp

catalog = intake.open_catalog("./*.yml")

facilities = gpd.read_parquet("./data/plot_facilities.parquet")
hqta_corr = gpd.read_parquet("./data/plot_hqta.parquet")

In [ ]:
hqta_popup = {
    "hqta_type": "HQTA Type",
    "agency_name_primary": "Primary Agency",
    "agency_name_secondary": "Secondary Agency",
}

# plot_col needs to be numeric. categorical throws error.
# try it with dtype category?
hqta_plot_col = "sqft"
color_hqta = branca.colormap.StepColormap(
    colors=[cp.CALITP_CATEGORY_BRIGHT_COLORS[0]]
)


facilities_popup = {
    "facility_name": "Name",
    "address_arcgis_clean": "Address" ,
    "category": "Facility Category",
    "facility_type": "Facility Type"
}

facilities_plot_col = "sqft"
color_facilities = branca.colormap.StepColormap(
    colors=["black"]
)

LAYERS_DICT = { 
    "hqta": {
        "df": hqta_corr,
        "plot_col": hqta_plot_col,
        "popup_dict": hqta_popup,
        "tooltip_dict": hqta_popup,
        "colorscale": color_hqta,
        "style_function": lambda x: {
            "fillColor": color_hqta(x["properties"][hqta_plot_col])
                if x["properties"][hqta_plot_col] is not None
                else "gray",
                "color": "#FFFFFF",
                "fillOpacity": 0.3,
                "weight": 0.2,
        }
    },
    "facilities": {
        "df": facilities,
        "plot_col": facilities_plot_col,
        "popup_dict": facilities_popup,
        "tooltip_dict": facilities_popup,
        "colorscale": color_facilities,
     # https://stackoverflow.com/questions/50954840/displaying-radius-in-meters-with-folium
     # Circle shows radius in pixels, CircleMarker shows radius in meters (or whatever CRS is set) 
        "marker": folium.Circle(radius=500, fill_color="black", 
                             fill_opacity=0.9, 
                             color="black", weight=2),
     "highlight_function": lambda x: {"fillOpacity": 0.8},
     "zoom_on_click": True
    },   
}

FIG_WIDTH = 500
FIG_HEIGHT = 800

In [ ]:
make_map.make_folium_multiple_layers_map(
    LAYERS_DICT,
    FIG_WIDTH,
    FIG_HEIGHT,
    zoom=map_utils.REGION_CENTROIDS["CA"]["zoom"],
    centroid=map_utils.REGION_CENTROIDS["CA"]["centroid"],
    title="Chart Title",
    legend_dict={"legend_url": "", "legend_bottom": 85, "legend_left": 5},
)